In [31]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import random

#####données à renseigner#####################

# URL de base pour le produit
base_url = 'https://fr.trustpilot.com/review/certideal.com'
#nombre de pages d'avis 
nombre_page = 500

################################################
# Initialisation de la session
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Referer': base_url
}

#############  déclaration variables tableau  ####################################
urls = []
page = []
phone = []
date_avis = []
titre_commentaires = []
commentaires = []
nom_client = []
lienavis = []
nb_avis_global = []
note_globale = []
note_client = []
nb_avis = []
notes = []
jours = []
pays = []
nom_entreprise = []

# une valeur par défaut pour les listes
default_value = None

############################### programme     ########################################

# Boucle pour parcourir les pages d' avis
for page in range(1, nombre_page):
    # Modification de l'URL pour changer le numéro de page
    url = f"{base_url}?page={page}"
    print(url)
    
    # Pause aléatoire entre les requêtes
    time.sleep(random.uniform(2, 5))  
    
    response = session.get(url, headers=headers)  # Utiliser la session avec les en-têtes
    if response.status_code != 200:
        print(f"Erreur lors de la récupération de la page {page}: {response.status_code}")
        break
    
    soup = bs(response.content, "lxml")

    # Nom entreprise
    company_name_element = soup.find('span', class_="typography_display-s__qOjh6 typography_appearance-default__AAY17 title_displayName__TtDDM")
    if company_name_element:
        company_name = company_name_element.get_text(strip=True)
        print(company_name)
    else:
        print("Nom de l'entreprise non trouvé.")
        company_name = default_value  # ou une autre valeur par défaut

    # Récupération de la note entreprise
    note_ent = soup.find('span', class_='typography_heading-m__T_L_X typography_appearance-default__AAY17').text.strip().replace(',', '.')

    # Récupération du nombre d'avis sur l'entreprise
    nb_note_ent = soup.find('p', class_='typography_body-l__KUYFJ typography_appearance-default__AAY17').text.strip().split(':')[-1].strip()
    nb_note_ent = nb_note_ent.replace('\u202f', '').replace(' ', '')

    # Recherche des avis dans la base
    base_avis = soup.find_all('div', class_="styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ")

    # Vérification si des avis ont été trouvées
    if not base_avis:
        print(f"Aucune critique trouvée sur la page {page}.")
        break

    for avis in base_avis:
        #page url      
        try:
            urls.append(url) 
        except AttributeError:
            urls.append(default_value)

        #nom entreprise     
        try:
            nom_entreprise.append(company_name) 
        except AttributeError:
            nom_entreprise.append(default_value)

        #note globale entreprise
        try:
            note_globale.append(note_ent) 
        except AttributeError:
            note_globale.append(default_value)

        #nombre total d'avis sur l'entreprise
        try:
            nb_avis_global.append(nb_note_ent) 
        except AttributeError:
            nb_avis_global.append(default_value)

        # Nom de la personne
        try:
            personne = avis.find('span', class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text.strip()
            nom_client.append(personne)
        except AttributeError:
            nom_client.append(default_value)

        #les titre_commentaires
        try:
            titre1 = avis.find('h2', class_='typography_heading-s__f7029 typography_appearance-default__AAY17').text.replace('\n', '').strip()
            titre_commentaires.append(titre1)
        except AttributeError:
            titre_commentaires.append(default_value)
        
        #les commentaires 
        try:
            commentaire1 = avis.find('p', class_='typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn')
            commentaire2 = commentaire1.get_text(strip=True)
            commentaires.append(commentaire2)
        except AttributeError:
            commentaires.append(default_value)

        #Date de publication
        try:
            date1 = avis.find('p', class_='typography_body-m__xgxZ_ typography_appearance-default__AAY17').text.strip('()')
            date_value = date1.split(': ')[1]
            date_avis.append(date_value)
        except AttributeError:
            date_avis.append(default_value)

        #nombre d'avis
        try:
            nb_avis1 = avis.find('div', class_='styles_consumerExtraDetails__fxS4S').get('data-consumer-reviews-count')
            nb_avis.append(nb_avis1)
        except AttributeError:
            nb_avis.append(default_value)

        #la période en jours
        try:
            jour1 = avis.find('div', class_='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_datesWrapper__RCEKH').get_text()
            jours.append(jour1)
        except AttributeError:
            jours.append(default_value)

        #la note
        try:
            note1 = avis.find('div', class_='styles_reviewHeader__iU9Px')['data-service-review-rating']
            notes.append(note1)
        except AttributeError:
            notes.append(default_value)

        #le pays
        try:
            pays1 = avis.find('div', class_='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua').find('span').text
            pays.append(pays1)
        except AttributeError:
            pays.append(default_value)

################# Création du DataFrame ###########################################################################

df_avis_certideal = pd.DataFrame({
    'Nom entreprise': nom_entreprise,
    'Page': urls,
    'Note globale': note_globale,
    'Nombre total avis': nb_avis_global,
    'Nom du client': nom_client,
    'Pays': pays,
    'Date de publication': date_avis,
    "Nombre d'avis du client": nb_avis,
    'Note client': notes,
    'Titre commentaire': titre_commentaires,
    'Commentaire': commentaires
})

# Afficher les 15 premières lignes du DataFrame
display(df_avis_certideal.head(15))

# Exporter le DataFrame en fichier CSV
df_avis_certideal.to_csv('/Users/yassinetazit/Desktop/Satisfaction Client/df_certideal.csv', index=False, encoding='utf-8')


https://fr.trustpilot.com/review/certideal.com?page=1
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=2
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=3
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=4
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=5
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=6
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=7
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=8
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=9
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=10
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=11
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=12
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=13
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=14
CertiDeal
https://fr.trustpilot.com/review/certideal.com?page=15
CertiDeal
https://fr.trustpilot.com/review/c

,Nom entreprise,Page,Note globale,Nombre total avis,Nom du client,Pays,Date de publication,Nombre d'avis du client,Note client,Titre commentaire,Commentaire
0,CertiDeal,https://fr.trustpilot.com/review/certideal.com...,4.4,26139,User,FR,14 janvier 2025,1,5,J'ai eu une super expérience avec le…,J'ai eu une super expérience avec le service a...
1,CertiDeal,https://fr.trustpilot.com/review/certideal.com...,4.4,26139,Harka Singh,FR,09 janvier 2025,1,5,Superbe expérience,"Étonnamment surprise, au départ j’étais très r..."
2,CertiDeal,https://fr.trustpilot.com/review/certideal.com...,4.4,26139,Katia,FR,11 janvier 2025,5,1,Téléphoné partiellement HS,Téléphone arrivé en état plus que bof bof car ...
3,CertiDeal,https://fr.trustpilot.com/review/certideal.com...,4.4,26139,Herlove,FR,20 décembre 2024,1,5,Service au top,Service au top !J’ai appelé le SAV car j’avais...
4,CertiDeal,https://fr.trustpilot.com/review/certideal.com...,4.4,26139,Salma Cavaignac,FR,01 octobre 2024,1,5,Téléphone,J’ai passé commande y’a un petit moment mainte...
5,CertiDeal,https://fr.trustpilot.com/review/certideal.com...,4.4,26139,MARIE FOLLET,FR,30 décembre 2024,8,1,Extrêmement déçue,Extrêmement déçue . Nous avons commandé 3 télé...
6,CertiDeal,https://fr.trustpilot.com/review/certideal.com...,4.4,26139,Arshdeep,FR,14 janvier 2025,1,5,J’ai passé commande d’un téléphone…,J’ai passé commande d’un téléphone recondition...
7,CertiDeal,https://fr.trustpilot.com/review/certideal.com...,4.4,26139,AURELIE MO,FR,10 janvier 2025,1,5,J'ai vraiment été impressionné par la…,J'ai vraiment été impressionné par la qualité ...
8,CertiDeal,https://fr.trustpilot.com/review/certideal.com...,4.4,26139,NBL,FR,11 janvier 2025,4,5,Tout s'est très bien passé,Tout s'est très bien passé. De la commande à l...
9,CertiDeal,https://fr.trustpilot.com/review/certideal.com...,4.4,26139,Shana,FR,15 janvier 2025,1,5,J'ai récemment acheté un téléphone…,J'ai récemment acheté un téléphone recondition...
